In [9]:
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [10]:
MODEL_NAME = "tdrenis/finetuned-bot-detector"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

In [15]:
with open("C:/Users/Bence/Documents/GitHub/Social-media-filter/data/comments.json", "r", encoding="utf-8") as f:
    comments = json.load(f)
inputs = tokenizer(comments, padding=True, truncation=True, return_tensors='pt')

In [ ]:
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

In [ ]:
labels = ["human", "bot"]  # Az osztályok, lehet, hogy módosítani kell a modell alapján
results = [{"comment": comment, "prediction": labels[pred.item()]} for comment, pred in zip(comments, predictions)]


In [ ]:
with open("bot_detection_results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

In [5]:
import torch
MODEL_NAME = "tdrenis/finetuned-bot-detector"

# Ellenőrizd, hogy van-e CUDA (GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modell és tokenizer betöltése
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
model.to(device)  # <- GPU-ra helyezi a modellt

# Szöveg előfeldolgozás és predikció
inputs = tokenizer("this message is generated by an ai bot", return_tensors="pt", truncation=True, padding=True)
inputs = {k: v.to(device) for k, v in inputs.items()}  # <- Bemenetek GPU-ra helyezése

with torch.no_grad():
    outputs = model(**inputs)

tensor([[-0.0994, -0.0423]])
